In [ ]:
%pylab inline

# sourmash algorithms and implementation

topics:
    
* modulo hash, "density hashing",
    * https://github.com/dib-lab/sourmash/issues/823
    * https://github.com/dib-lab/sourmash/issues/606
    * https://github.com/richarddurbin/modimizer
* 'scaled' implementation, and using scaled hashes for Jaccard similarity and containment calculations


In [7]:
import screed

def kmerize_seq(sequence, k=31):
    sequence = sequence.upper()
    for start in range(0, len(sequence) - k + 1):
        kmer = sequence[start:start+k]
        
        # canonicalize the k-mer
        revcomp = screed.rc(kmer)
        if kmer < revcomp:
            yield kmer
        else:
            yield revcomp
            
#list(kmerize('ATGGACAGAGATG', k=2))

def kmerize_file(filename, k=31):
    # walk through every record in a FASTA/FASTQ file
    for record in screed.open(filename):
        # k-merize each sequence
        for kmer in kmerize_seq(record.sequence[:50000]):
            # return canonical version of each k-mer
            yield kmer

len(set(kmerize_file('data/genomes/2.fa')))



49970

In [8]:
import mmh3

In [25]:
import mmh3

# implementation of a minhash bottom sketch, where a fixed number
# of hashes is kept for each input set of k-mers.

# there's a bug in this code where we are not handling duplicate k-mers
# properly.
def minhash(kmers, num=500):
    basket = []
    for kmer in kmers:
        hashval = mmh3.hash64(kmer, seed=42)[0]
        if hashval < 0:
            hashval += 2**64
        
        # basket not full? add hash value
        if len(basket) < num:
            basket.append(hashval)
            basket = list(sorted(basket))
        # basket full?
        elif len(basket) == num:
            if basket[-1] > hashval:
                # this hashvalue does not belong in basket
                pass
            else:
                # is new hash value less than largest? if so, evict.
                basket.pop()
                basket.append(hashval)
                basket = list(sorted(basket))
                
    return basket

b_all = set(kmerize_file('data/genomes/2.fa'))
c_all = set(kmerize_file('data/genomes/47.fa'))
d_all = set(kmerize_file('data/genomes/63.fa'))


In [29]:
b = minhash(b_all, num=1000)
c = minhash(c_all, num=1000)
d = minhash(d_all, num=1000)


In [30]:
print(len(b_all), len(c_all), len(d_all))
print(len(b), len(c), len(d))

49970 96808 169296
1000 1000 1000


In [31]:
print(type(b_all))
print(type(b))

<class 'set'>
<class 'list'>


In [32]:
def jaccard_similarity(x, y):
    x = set(x)
    intersection = x.intersection(y)
    union = x.union(y)
    return len(intersection) / len(union)

print('b_all to b_all', jaccard_similarity(b_all, b_all))
print('b to b', jaccard_similarity(b, b))

print('b_all to c_all', jaccard_similarity(b_all, c_all))
print('b to c', jaccard_similarity(b, c))

print('c_all to d_all', jaccard_similarity(c_all, d_all))
print('c to d', jaccard_similarity(c, d))

b_all to b_all 1.0
b to b 1.0
b_all to c_all 0.0
b to c 0.0
c_all to d_all 0.2544146624303506
c to d 0.1607661056297156


In [33]:
# now instead of a minhash, implement a scaled sketch, which is what
# we recommend using in sourmash.

# scaled here is 1/f of k-mers that we are going to keep
# this is a sampling mechanism where we "stochastically" sample
# by randomizing the order of k-mers with a hash function, and
# then choosing a subset of them deterministically.

def scaledhash(kmers, scaled=1000):
    basket = set()
    
    MAX_HASH=2**64 - 1
    boundary = MAX_HASH / scaled
    
    for kmer in kmers:
        hashval = mmh3.hash64(kmer, seed=42)[0]
        if hashval < 0:
            hashval += 2**64
            
        # accept any k-mer that hashes into the bottom 1/scaled of the
        # hash space. what this should do (for a good hash function)
        # is pick approximately 1 in 'scaled' of the input k-mers.
        if hashval < boundary:
            basket.add(hashval)

    return basket
        
b_sh = scaledhash(b_all, scaled=1000)
c_sh = scaledhash(c_all, scaled=1000)
d_sh = scaledhash(d_all, scaled=1000)


In [34]:
print('b_all to b_all', jaccard_similarity(b_all, b_all))
print('b to b', jaccard_similarity(b, b))
print('b_sh to b_sh', jaccard_similarity(b_sh, b_sh))


print('b_all to c_all', jaccard_similarity(b_all, c_all))
print('b to c', jaccard_similarity(b, c))
print('b_sh to c_sh', jaccard_similarity(b_sh, c_sh))

print('c_all to d_all', jaccard_similarity(c_all, d_all))
print('c to d', jaccard_similarity(c, d))
print('c_sh to dsh', jaccard_similarity(c_sh, d_sh))

b_all to b_all 1.0
b to b 1.0
b_sh to b_sh 1.0
b_all to c_all 0.0
b to c 0.0
b_sh to c_sh 0.0
c_all to d_all 0.2544146624303506
c to d 0.1607661056297156
c_sh to dsh 0.1889400921658986



* 'scaled' implementation, and using scaled hashes for Jaccard similarity and containment calculations
    * with a scaled of 10,000, you get approximately 1 hash per every 10kb of sequence
    * see [graph at 10kb](https://github.com/dib-lab/charcoal/blob/master/stats/stats10k.png), [graph at 5kb](https://github.com/dib-lab/charcoal/blob/master/stats/stats5k.png)
    * (similar statistics apply on a stream of distinct k-mers)


* looking in more depth at the signatures
* manipulating signatures (command line, Python, etc.)


In [36]:
!jq < 2.fa.sig

[
  {
    "class": "sourmash_signature",
    "email": "",
    "hash_function": "0.murmur64",
    "filename": "data/genomes/2.fa",
    "name": "CP001071.1 Akkermansia muciniphila ATCC BAA-835, complete genome",
    "license": "CC0",
    "signatures": [
      {
        "num": 0,
        "ksize": 21,
        "seed": 42,
        "max_hash": 18446744073709550,
        "mins": [
          9253370411757,
          22055233311447,
          26285766970543,
          27296987485978,
          30361772373880,
          31035807336206,
          34747011898208,
          41267394135989,
          55336461597857,
          58422622892529,
          64979457479652,
          73153444791484,
          75464211763099,
          85635123992283,
          92061589746066,
          94676917797247,
          112004811147472,
          114131388616210,
          123015037473295,
          128409344691820,
          139465367894647,
          162229727270337,
          164843936183953,
          1693177810

          11106426173096134,
          11111008680574732,
          11112971544804072,
          11117142906970704,
          11122887944491864,
          11123134489672404,
          11124313190051970,
          11131274045814432,
          11134768830434580,
          11137871193966872,
          11139439324849368,
          11145533100200114,
          11150515087815276,
          11156384085429662,
          11157892369956176,
          11161009408719874,
          11166211799466712,
          11170717557392304,
          11184536722543476,
          11187525741338652,
          11226007334467712,
          11236648797562228,
          11238092844710720,
          11239363750653622,
          11247734327298832,
          11261039636130716,
          11261116396288180,
          11261405867666512,
          11262384370067150,
          11263691442159888,
          11269376706100644,
          11280308878926292,
          11281981040144194,
          11290418422286480,
          1129

          5703741162189807,
          5706535934081315,
          5707128466246152,
          5717731977260371,
          5719633091855226,
          5720510963194346,
          5729579279104706,
          5729764028324178,
          5740630525074950,
          5741129486732745,
          5745453911718457,
          5749789577819561,
          5752026401606325,
          5760229237068668,
          5762776817176351,
          5767566225687055,
          5770214575183328,
          5771595211018276,
          5781390291816527,
          5804083696129527,
          5808000639892486,
          5816457793684100,
          5826243474287654,
          5852002021836045,
          5857735946215711,
          5875027588671735,
          5875545612812880,
          5880265717894672,
          5881862761884521,
          5882088106486161,
          5890217597925952,
          5891438242420236,
          5893381947716218,
          5895285724492612,
          5907529378335018,
          5907926678

          3821370728424399,
          3831091291497756,
          3833594319775588,
          3836070293893334,
          3836463726302801,
          3843454534654819,
          3865330015850533,
          3865731389902624,
          3868815096902378,
          3874152187538165,
          3879504507792185,
          3881012401310194,
          3890928823553932,
          3894340812924690,
          3910525773502689,
          3913041691319282,
          3915621729619021,
          3919374357408690,
          3957513182725671,
          3959863515871547,
          3964876524010791,
          3969782925917955,
          3991647820720584,
          3999166703998717,
          4002558494513051,
          4017396410306556,
          4027670709113821,
          4031031935378161,
          4039069154849114,
          4054564391899400,
          4055069780054908,
          4063620726694105,
          4074966284721173,
          4078778415329361,
          4083514031241394,
          4083640661

          14745918381531036,
          14747781274261754,
          14769235813387444,
          14774506216866144,
          14775858199845320,
          14777438862619136,
          14785706151846816,
          14797080542518334,
          14808067053963172,
          14815595921590586,
          14816883728169456,
          14819323611785348,
          14819840263469102,
          14827625495688744,
          14839029876559500,
          14850591011118274,
          14861741509747314,
          14892690229161784,
          14894257791973640,
          14902573036352340,
          14913146370699988,
          14926907949184832,
          14932875610139882,
          14934335561729208,
          14936625799272706,
          14952709199705300,
          14953000360859160,
          14956638768476526,
          14965904599452928,
          14966257876394972,
          14977525168387132,
          14978894796691800,
          14983094219624524,
          14984259611333822,
          1498

          17825363186790720,
          17839438010001086,
          17854602718146652,
          17859786785776348,
          17877311442819478,
          17905061698046516,
          17915900664445324,
          17921723325622658,
          17927897596423856,
          17945154459135646,
          17946171820546868,
          17963522321206264,
          17968127380401234,
          17969920306213012,
          17970588515711704,
          17971561735516464,
          17979603471308842,
          17981246765331158,
          17991312557003936,
          17993515278583148,
          17994374617688536,
          18000259557770972,
          18025967626901544,
          18031442833041870,
          18038451019408690,
          18045491360368656,
          18054372143801230,
          18057809078702680,
          18058451040919700,
          18066894178440520,
          18081251148350710,
          18084278558782556,
          18088253811209544,
          18090491977476636,
          1809

## repo at github.com/ctb/2020-jgi-tech-talk

In [40]:
import sourmash
sig2 = sourmash.load_one_signature('2.fa.sig', ksize=31)
sig63 = sourmash.load_one_signature('63.fa.sig', ksize=31)
sig47 = sourmash.load_one_signature('47.fa.sig', ksize=31)

In [42]:
sig63.similarity(sig47)

0.3206949023586102

In [43]:
a = set(sig63.minhash.get_mins())
b=  set(sig47.minhash.get_mins())

jaccard_similarity(a, b)

0.3206949023586102

In [44]:
!sourmash search 47.fa.sig 63.fa.sig


== This is sourmash version 3.2.4.dev5+g6484e78f. ==
== Please cite Brown and Irber (2016), doi:10.21105/joss.00027. ==

selecting default query k=31.
loaded query: NC_009665.1 Shewanella baltica... (k=31, DNA)
loaded 1 signatures.                                                           

1 matches:
similarity   match
----------   -----
 32.1%       NC_011663.1 Shewanella baltica OS223, complete genome


* downsampling in particular
* ...hash matches do indeed correspond to nucleotide alignments!
* downsides of scaled approach:
    * don't work for small genomes
    * arbitrary growth in size (so may be unnecessarily large for large genomes!)
    
tl;dr sourmash is basically many thousands of lines of code around doing this stuff with reasonable efficiency and care.

In [45]:
sig63.similarity(sig47)

0.3206949023586102

In [46]:
sig63.contained_by(sig47)

0.48281786941580757

The scaledhash represents a compositional approach to sketching.

In [47]:
d_sh

{71315379183989,
 189372881188038,
 312508185357838,
 385242214102144,
 395356088720884,
 444196295289556,
 457671098462976,
 623564286058606,
 717098217279451,
 736734785664407,
 792800620274218,
 1075576974119467,
 1127740241225995,
 1278259882696625,
 1339674524726757,
 1432423484009425,
 1466391743844010,
 1505096985478881,
 1535451811798960,
 1596940098556068,
 1642353838656017,
 1658104204669725,
 1669812089421733,
 1719016742757524,
 1800236363852916,
 1911464021342251,
 1977965122461301,
 1996672784729607,
 2154825108832254,
 2296330477067902,
 2359599974602456,
 2556506799873846,
 2566619437831302,
 2742141972024096,
 2798287443905216,
 2823296594306435,
 2947035710205399,
 3009879725684408,
 3056598953438835,
 3101693309309556,
 3103723994300125,
 3173337239261818,
 3315214714717367,
 3320924500756368,
 3321523572005253,
 3549947024705822,
 3675775119596083,
 3863738703644495,
 4421125787216995,
 4473513246905919,
 4499856296475689,
 4501876131664304,
 4607450449118254,
 4613

In [57]:
# note you can downsample scaled hash signatures without going back to 
# the raw data.
def downsample_scaledhash(input_basket, scaled=1000):
    basket = set()
    
    MAX_HASH=2**64 - 1
    boundary = MAX_HASH / scaled
    
    for hashval in input_basket:
        if hashval < 0:
            hashval += 2**64
            
        # accept any k-mer that hashes into the bottom 1/scaled of the
        # hash space. what this should do (for a good hash function)
        # is pick approximately 1 in 'scaled' of the input k-mers.
        if hashval < boundary:
            basket.add(hashval)

    return basket


In [53]:
b_sh2 = downsample_scaledhash(b_sh, 1000)
b_sh2 == b_sh
b_sh3 = downsample_scaledhash(b_sh, 10000)

In [54]:
len(b_sh3)

9

In [55]:
len(b_sh2)

52